In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [3]:
#custom class for our NN
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(), #use ReLU for non-linearity
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10), #10 possible outputs
        )

#define the operations on the input data
# called when passing input data to model
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model) #have a look at the model and layers

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
# how is the model prdicting anything, when was it trained on any data?

X = torch.rand(1, 28, 28, device=device)
# print(X)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8])


In [ ]:
#following blocks of code can be usefull for looking into out models and debugging

In [9]:
# take batch of 3 random images
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [10]:
#flatten converts each image into 28*28 array
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [11]:
# apply a linear transformation, indicating number of input and output features
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [12]:
#ReLU is very important to add non-lenarity to the model, can also use tanh
# (-)ve values are set to 0
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1573,  0.3349,  0.2907,  0.2788,  0.2483,  0.0082, -0.0427,  0.2401,
          0.4767,  0.2821,  0.4041,  0.0489, -0.2386, -0.3540, -0.2727,  0.6303,
         -0.2354, -0.2900,  0.5424,  0.7738],
        [-0.2450,  0.3751,  0.5492,  0.6769,  0.3454,  0.3176, -0.0087,  0.1954,
          0.3705,  0.2990,  0.5254,  0.2501, -0.3297, -0.2894, -0.3129,  0.6802,
          0.0085, -0.1415,  0.3631,  0.6148],
        [ 0.0058,  0.7144,  0.0984,  0.2335,  0.3670,  0.1217, -0.0554,  0.0404,
          0.1632,  0.2858,  0.1970,  0.1805, -0.2480, -0.6764, -0.5322,  0.6440,
         -0.1253, -0.5909,  0.1619,  0.6426]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1573, 0.3349, 0.2907, 0.2788, 0.2483, 0.0082, 0.0000, 0.2401, 0.4767,
         0.2821, 0.4041, 0.0489, 0.0000, 0.0000, 0.0000, 0.6303, 0.0000, 0.0000,
         0.5424, 0.7738],
        [0.0000, 0.3751, 0.5492, 0.6769, 0.3454, 0.3176, 0.0000, 0.1954, 0.3705,
         0.2990, 0.5254, 0.2501, 0.0000, 0.0000, 0.00

In [13]:
#very similar to the Sequential API for TensorFlow
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [19]:
#logits are in [-inf, inf], passed to Softmax to scale to probability
# `dim` indicates the dimension where sum should be 1 dim[0,1,2,...]
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.1018, 0.0907, 0.1102, 0.0976, 0.0855, 0.1226, 0.1067, 0.0820, 0.0955,
         0.1074],
        [0.1012, 0.0961, 0.1104, 0.0976, 0.0775, 0.1178, 0.1033, 0.0819, 0.1035,
         0.1106],
        [0.1199, 0.0975, 0.1050, 0.1091, 0.0661, 0.1255, 0.1015, 0.0734, 0.0955,
         0.1065]], grad_fn=<SoftmaxBackward0>)

In [21]:
#checking out the model parameters

In [20]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0135,  0.0076, -0.0122,  ..., -0.0051, -0.0117,  0.0283],
        [-0.0143, -0.0165, -0.0203,  ..., -0.0017, -0.0314, -0.0104]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0205, 0.0296], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0263,  0.0423,  0.0069,  ...,  0.0022, -0.0200, -0.0436],
        [-0.0075,  0.0140, -0.0237,  ..., -0.0073,  0.0302,  0.0016]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si